---
## Versão 0.1  
* Split do treino (80/20).
* Features removidas: `Cabin`, `PassengerId`, `Name` e `Ticket`.
* Features `Age` e `Fare`: missing values preenchidos pela média.
* Feature `Embarked`: missing values preenchidos pelo mais comum.
* Features categóricas tratadas com `get_dummies` do `pandas`.
* Modelo: `DecisionTreeClassifier(max_depth=3, random_state=0)`

**Resultado no Kaggle**: 0.79425

---

### **Módulos**

In [2]:
import pandas as pd
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix

### **Datasets**

Primeiro importamos os datasets originais.

In [3]:
treino = pd.read_csv('./datasets/train.csv')
holdout_ini = pd.read_csv('./datasets/test.csv')

Como não temos o target do dataset de teste(holdout), faremos um split do dataset de treino a fim de poder avaliar o resultado da predição.  
Adotaremos a seguinte proporção: 80% treino e 20% teste.

In [4]:
# Separa "features"(X) e "target"(y)
feat = treino.drop('Survived', axis=1)
targ = treino['Survived']

# Faz o split
X_treino_ini, X_teste_ini, y_treino, y_teste = train_test_split(feat, targ, test_size=0.2, random_state=0)

A fim de podermos efetuar alterações sem gerar erros, faremos uma cópia dos splits com as features.

In [5]:
X_treino = X_treino_ini.copy()
X_teste = X_teste_ini.copy()
holdout = holdout_ini.copy()

Verificação de cada parte dos datasets a fim de garantir que os tamanho são compatíveis.

In [6]:
print('Treino:', X_treino.shape, y_treino.shape)
print('Teste:', X_teste.shape, y_teste.shape)
print('Holdout:', holdout.shape)

Treino: (712, 11) (712,)
Teste: (179, 11) (179,)
Holdout: (418, 11)


### **Análise e filtragem das features**

In [7]:
X_treino.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
140,141,3,"Boulos, Mrs. Joseph (Sultana)",female,NaN,0,2,2678,15.2458,NaN,C
439,440,2,"Kvillner, Mr. Johan Henrik Johannesson",male,31.0,0,0,C.A. 18723,10.5000,NaN,S
817,818,2,"Mallet, Mr. Albert",male,31.0,1,1,S.C./PARIS 2079,37.0042,NaN,C
378,379,3,"Betros, Mr. Tannous",male,20.0,0,0,2648,4.0125,NaN,C
491,492,3,"Windelov, Mr. Einar",male,21.0,0,0,SOTON/OQ 3101317,7.2500,NaN,S


Inicialmente analisaremos se alguma feature tem pouca amostragem e merece ser descartada.

In [8]:
na = pd.DataFrame()
na['treino'] = (X_treino.isna().sum() / len(X_treino.index) * 100)
na['teste'] = (X_teste.isna().sum() / len(X_teste.index) * 100)
na['holdout'] = (holdout.isna().sum() / len(holdout.index) * 100)
na.sort_values(['treino','teste','holdout'] ,ascending=False)

,treino,teste,holdout
Cabin,77.106742,77.094972,78.229665
Age,19.803371,20.111732,20.574163
Embarked,0.280899,0.000000,0.000000
Fare,0.000000,0.000000,0.239234
PassengerId,0.000000,0.000000,0.000000
Pclass,0.000000,0.000000,0.000000
Name,0.000000,0.000000,0.000000
Sex,0.000000,0.000000,0.000000
SibSp,0.000000,0.000000,0.000000
Parch,0.000000,0.000000,0.000000


Notamos que `Cabin` tem mais de 77% de valores faltantes em todo o dataset.   
Como a quantidade é muito grande, tentar completar esses valores pode gerar muita imprecisão. Sendo assim, descartaremos essa feature.

In [9]:
# "Cabin" tem mais de 77% de valores faltantes em todo o dataset. Vamos descartá-la.
X_treino.drop('Cabin', axis=1, inplace=True)
X_teste.drop('Cabin', axis=1, inplace=True)
holdout.drop('Cabin', axis=1, inplace=True)

A feature `PassengerId` é única por registro e não contém nenhuma informação relevante. Vamos descartá-la.

In [10]:
X_treino.drop('PassengerId', axis=1, inplace=True)
X_teste.drop('PassengerId', axis=1, inplace=True)
holdout.drop('PassengerId', axis=1, inplace=True)

Para simplificar esse primeiro estudo, descartaremos também as features `Name` e `Ticket`, que não apresentam nenhum informação majoritariamente relevante.

In [11]:
X_treino.drop(['Name','Ticket'], axis=1, inplace=True)
X_teste.drop(['Name','Ticket'], axis=1, inplace=True)
holdout.drop(['Name','Ticket'], axis=1, inplace=True)

Então temos o seguinte conjunto de features a ser inserido no modelo.

In [12]:
X_treino.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
140,3,female,NaN,0,2,15.2458,C
439,2,male,31.0,0,0,10.5000,S
817,2,male,31.0,1,1,37.0042,C
378,3,male,20.0,0,0,4.0125,C
491,3,male,21.0,0,0,7.2500,S


Contudo, primeiro é preciso garantir que os dados estejam consistentes e num formato que seja aceito pelo modelo.

### **Tratamento de "Missing Values"**

In [13]:
na = pd.DataFrame()
na['treino'] = (X_treino.isna().sum() / len(X_treino.index) * 100)
na['teste'] = (X_teste.isna().sum() / len(X_teste.index) * 100)
na['holdout'] = (holdout.isna().sum() / len(holdout.index) * 100)
na['tipo'] = X_treino.dtypes
na.sort_values(['treino','teste','holdout'] ,ascending=False)

,treino,teste,holdout,tipo
Age,19.803371,20.111732,20.574163,float64
Embarked,0.280899,0.000000,0.000000,object
Fare,0.000000,0.000000,0.239234,float64
Pclass,0.000000,0.000000,0.000000,int64
Sex,0.000000,0.000000,0.000000,object
SibSp,0.000000,0.000000,0.000000,int64
Parch,0.000000,0.000000,0.000000,int64


Notamos que `Age`, `Embarked` e `Fare` tem valores faltantes.

`Age` e `Fare` são features numéricas. Completaremos com a média dos demais.

In [14]:
X_treino['Age'].fillna(X_treino['Age'].mean(), inplace=True)
X_teste['Age'].fillna(X_teste['Age'].mean(), inplace=True)
holdout['Age'].fillna(holdout['Age'].mean(), inplace=True)
holdout['Fare'].fillna(holdout['Fare'].mean(), inplace=True)

`Embarked` é uma feature categórica. Preencheremos os faltantes com o valor mais comum.

In [15]:
X_treino['Embarked'].fillna(X_treino['Embarked'].value_counts().index[0], inplace=True)

Agora não temos mais valores faltantes em nenhuma feature.

In [16]:
na['treino'] = (X_treino.isna().sum() / len(X_treino.index) * 100)
na['teste'] = (X_teste.isna().sum() / len(X_teste.index) * 100)
na['holdout'] = (holdout.isna().sum() / len(holdout.index) * 100)
na['tipo'] = X_treino.dtypes
na.sort_values(['treino','teste','holdout'] ,ascending=False)

,treino,teste,holdout,tipo
Pclass,0.0,0.0,0.0,int64
Sex,0.0,0.0,0.0,object
Age,0.0,0.0,0.0,float64
SibSp,0.0,0.0,0.0,int64
Parch,0.0,0.0,0.0,int64
Fare,0.0,0.0,0.0,float64
Embarked,0.0,0.0,0.0,object


### **Tratamento de features categóricas**

Como os modelos só tratam números, precisamos converter as features categóricas.  
Para tal, usaremos a função `get_dummies` do `pandas`.  
Cada feature categórica terá seus valores únicos separados em uma nova coluna exclusiva e as originais deixarão de existir.

In [17]:
X_treino.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
140,3,female,29.745184,0,2,15.2458,C
439,2,male,31.000000,0,0,10.5000,S
817,2,male,31.000000,1,1,37.0042,C
378,3,male,20.000000,0,0,4.0125,C
491,3,male,21.000000,0,0,7.2500,S


In [18]:
X_treino = pd.get_dummies(X_treino)
X_teste = pd.get_dummies(X_teste)
holdout = pd.get_dummies(holdout)
X_treino.head()

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
140,3,29.745184,0,2,15.2458,1,0,1,0,0
439,2,31.000000,0,0,10.5000,0,1,0,0,1
817,2,31.000000,1,1,37.0042,0,1,1,0,0
378,3,20.000000,0,0,4.0125,0,1,1,0,0
491,3,21.000000,0,0,7.2500,0,1,0,0,1


### **Definindo o modelo, treinando e avaliando o resultado**

<img src='./images/Machine-Learning-Classico.png' width='600' align='left'>

Com base na figura acima, aonde temos uma simplificação das principais classes de modelos, podemos identificar qual melhor caminho seguir na escolha do mais adequado para a solução do nosso problema.

Como em nosso caso os dados estão categorizados/rotulados(sabemos o que acontece com cada passageiro), concluímos que trata-se de uma **análise supervisionada**.  
Nessa situação, o modelo será responsável por achar uma "fórmula" que leve os dados apresentadas às respostas que já conhecemos.

Tendo em vista que vamos tentar prever uma categoria(passageiro sobreviveu ou não), e não um valor/grandeza, usaremos modelos de **classificação**.

Dentre as opções disponíveis, vamos começar com o [Decision Tree](https://towardsdatascience.com/decision-trees-in-machine-learning-641b9c4e8052), que é um modelo de fácil compreensão, que demanda pouco tratamento dos dados de entrada e que não precisa de configuração complexa de parâmetros.

In [28]:
modelo = DecisionTreeClassifier(max_depth=3, random_state=0)
modelo.fit(X_treino, y_treino)
modelo.score(X_treino, y_treino)

0.8342696629213483

Aqui podemos ver quais features o modelo deu mais importância.

In [30]:
feat_imp = modelo.feature_importances_
feat = pd.DataFrame(list(zip(X_treino,feat_imp)))
feat.columns = ['Feature','Importance']
feat.set_index('Feature', inplace=True)
feat.sort_values('Importance', ascending=False)

,Importance
Feature,
Sex_male,0.621926
Pclass,0.176426
SibSp,0.075520
Age,0.070737
Fare,0.055391
Parch,0.000000
Sex_female,0.000000
Embarked_C,0.000000
Embarked_Q,0.000000


### **Predição com o dataset de teste e avaliação**

Vamos agora pegar a segunda parte do split do dataset original e ver se o modelo já treinado consegue prever quais passageiros dessa amostra sobreviveram e quais não.

In [31]:
pred_teste = modelo.predict(X_teste)
print(pred_teste.shape)
pred_teste

(179,)


array([0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0,
       1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1,
       0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       1, 0, 0], dtype=int64)

Na amostra testada temos a seguinte distribuição de passageiros:

In [32]:
print('Falecidos:',y_teste.value_counts()[0])
print('Sobreviventes:',y_teste.value_counts()[1])

Falecidos: 110
Sobreviventes: 69


Analisando a [matriz de confusão](https://en.wikipedia.org/wiki/Confusion_matrix), podemos identificar a quantidade de erros e acertos em cada categoria.

In [33]:
print(' TP FN')
print(confusion_matrix(y_teste, pred_teste)[0], 'Falecidos')
print(confusion_matrix(y_teste, pred_teste)[1], 'Sobreviventes')
print(' FP TN')

 TP FN
[96 14] Falecidos
[18 51] Sobreviventes
 FP TN


O modelo previu:
* 96 falecidos corretamente
* 51 sobreviventes corretamente
* 14 falecidos como sobreviventes
* 18 sobreviventes como falecidos

Aplicando algumas [métricas de medição](https://gabrielschade.github.io/2019/03/12/ml-classificacao-metricas.html) podemos avaliar a precisão do resultado.

In [34]:
print(classification_report(y_teste, pred_teste, digits=4, target_names=['Falecidos', 'Sobreviventes']))

               precision    recall  f1-score   support

    Falecidos     0.8421    0.8727    0.8571       110
Sobreviventes     0.7846    0.7391    0.7612        69

     accuracy                         0.8212       179
    macro avg     0.8134    0.8059    0.8092       179
 weighted avg     0.8199    0.8212    0.8202       179



O modelo acertou cerca de 81% dos resultados. Sendo que foi mais eficiente na identificação de falecidos do que de sobreviventes.

### **Predição com o dataset de holdout**

Agora vamos repetir o processo de predição com a amostra de teste.

In [35]:
pred_holdout = modelo.predict(holdout)
print(pred_holdout.shape)
pred_holdout

(418,)


array([0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1,
       0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,

Como não temos os dados de sobreviventes/falecidos dessa amostra, não há com o que comparar.

### **Resultado para submissão**

Vamos então gerar um arquivo para submeter a nossa predição à avaliação do Kaggle e lá saberemos o quanto o modelo acertou dessa amostra.

In [26]:
resultado = pd.DataFrame({'PassengerId': holdout_ini['PassengerId'], 'Survived': pred_holdout})
resultado.set_index('PassengerId', inplace=True)
resultado.head()

,Survived
PassengerId,
892,0
893,1
894,0
895,0
896,1


In [27]:
t = datetime.now().strftime('%Y%m%d_%H%M')
resultado.to_csv(f'./submissions/titanic_v0.1_{t}.csv')

O resultado obtido foi:

<img src='./images/titanic_v01_20200601_1909.png' align='left'>